In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import shutil
import os

imported_dataset_path = '/kaggle/input/artificial-lunar-rocky-landscape-dataset'
destination_path = '/kaggle/working/artificial-lunar-rocky-landscape-dataset'

shutil.copytree(imported_dataset_path, destination_path)
print(os.listdir(destination_path))

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     print(dirname)
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['top200_largerocks_IDs.txt', 'bounding_boxes.csv', 'mismatch_IDs.txt', 'cam_anomaly_IDs.txt', 'shadow_IDs.txt', 'real_moon_images', 'ground_facing_IDs.txt', 'images']


In [3]:
import torch, torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.transforms import transforms
from torchmetrics.detection import MeanAveragePrecision
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm

Data Preprocessing

In [4]:
# creating basic classifier for rock sizes
class BasicClassifier():
    """
    Sorts images into 3 classes based on their area: 
    small_rock (1), medium_rock (2), large_rock (3)
    """
    
    def __init__(self, df: pd.DataFrame, img_width=720, img_height=480):
        self.df = df.copy()
        self.img_width = img_width
        self.img_height = img_height
        
        # un-normalizing box height/width to calculate absolute area
        self.df['Area'] = self.df['Length'] * self.df['Height']
        
        self.df = self.df.sort_values(by='Area', ascending=True)
        
        # split dataset into thirds
        df_length = len(self.df)
        self.lower_limit = self.df['Area'].iloc[df_length // 3 - 1] # first third
        self.middle_limit = self.df['Area'].iloc[2 * df_length // 3 - 1] # second third
        
    def classify_df(self):
        return self.df.apply(
            lambda row: self.classify_box(
                box_width = row['Length'],
                box_height = row['Height']
            ),
            axis=1 
        )

    def classify_box(self, box_width: float, box_height: float):
        box_area = box_width * box_height

        # categorize rock depending on size
        if box_area <= 0:
            return 0 # None
        elif 0 < box_area <= self.lower_limit:
            return 1 # Small Rock
        elif self.lower_limit < box_area <= self.middle_limit:
            return 2 # Medium Rocks
        else:
            return 3 # Large Rock




    # MISC: helpful dataset visualizer functions (below)

    # visualize distribution of bounding box sizes and help determine approx. class sizes
    def box_size_distr_visualizer(self):
        print("Pre-Filtering Values:")
        self.key_value_statistics(self.df['Area'])

        # Remove outliers using 1.5*IQR Rule
        AreaQ1 = self.df['Area'].quantile(0.25)
        AreaQ3 = self.df['Area'].quantile(0.75)
        area_iqr = AreaQ3 - AreaQ1
        iqr_for_q1 = AreaQ1 - 1.5 * area_iqr
        iqr_for_q3 = AreaQ3 + 1.5 * area_iqr

        df_cleaned = self.df['Area'][(self.df['Area'] >= iqr_for_q1) & (self.df['Area'] <= iqr_for_q3)]
        print("Post-Filtering Values:")
        self.key_value_statistics(df_cleaned)
        
        # Visualize dataset
        custom_bins = list(np.arange(0,10**4, 500))
        plt.hist(self.df['Area'], bins=custom_bins)
        plt.title('Area Distribution of Bounding Box Sizes')
        plt.xlabel('Box Area')
        plt.ylabel('Frequency')
        plt.show()
    
    # display key statistics for a list/column of values
    def key_value_statistics(self, values): 
        print(f"Max: {values.max()}. Min: {values.min()}. Median: {values.median()}. Mean: {values.mean()} ")

In [5]:
# RCNN: convert to x_min, y_min, x_max, y_max
def convert_to_rcnn(dataset_path: str, df: pd.DataFrame):
    new_df = pd.DataFrame()
    
    new_df['image'] = 'render' + df['Frame'].astype(int).astype(str).str.zfill(4) + '.png' # Image Name: 1 --> 00001
    new_df['x_min'] = df['TopLeftCornerX'].round(0).astype(int)
    new_df['y_min'] = df['TopLeftCornerY'].round(0).astype(int)
    new_df['x_max'] = (df['TopLeftCornerX'] + df['Length']).round(0).astype(int)
    new_df['y_max'] = (df['TopLeftCornerY'] + df['Height']).round(0).astype(int)
    
    classifier = BasicClassifier(df)
    new_df['class_id'] = classifier.classify_df()
    
    return new_df


In [6]:
# remove faulty images from dataset (October Kaggle update)
def remove_errors(dataset_path: str, df: pd.DataFrame):
    # faulty_path = os.path.join(dataset_path, 'faulty_images')
    txt_files = [i for i in os.listdir(dataset_path) if i.endswith('.txt')]
    faulty_images = []
    
    for txt in txt_files:
        with open(os.path.join(dataset_path, txt)) as file:
            for line in file:
                faulty_images.append(f"render{line.strip()}.png")
                
    # print(len(faulty_images)) # 773 faulty images exist
    img_path = os.path.join(dataset_path, 'images/render')
    
    # removes faulty images from render folder
    def delete_images(img_path: str, faulty_images: list):
        data_images = [img for img in os.listdir(img_path) if img.endswith('.png')]
        for img in data_images:
            if img in faulty_images:
                os.remove(os.path.join(img_path, img))

    # removes rows of faulty images from dataframe
    def delete_boxes(new_df: pd.DataFrame, faulty_images: list):
        filtered_df = new_df[~new_df['image'].isin(faulty_images)]
        return filtered_df
    
    delete_images(img_path, faulty_images)
    filtered_df = delete_boxes(df, faulty_images)
    return filtered_df


In [7]:
dataset_path = '/kaggle/working/artificial-lunar-rocky-landscape-dataset'
    
df_path = os.path.join(dataset_path, 'bounding_boxes.csv')
df = pd.read_csv(df_path)

new_df = convert_to_rcnn(dataset_path, df)

new_df = remove_errors(dataset_path, new_df)

new_df_path = os.path.join(dataset_path, 'rcnn_bounding_boxes_final.csv')
new_df.to_csv(new_df_path, index=False)

In [8]:
# Confirming csv file was added to kaggle/working
print(os.listdir(dataset_path))

# verifying df was cleaned
print("csv")
print(len(df))
print(len(new_df))

# verifying image folder was cleaned
print("Folder")
img_path = os.path.join(dataset_path, 'images/render')
print(len(os.listdir(img_path)))

['top200_largerocks_IDs.txt', 'bounding_boxes.csv', 'mismatch_IDs.txt', 'cam_anomaly_IDs.txt', 'shadow_IDs.txt', 'rcnn_bounding_boxes_final.csv', 'real_moon_images', 'ground_facing_IDs.txt', 'images']
csv
18867
16819
Folder
8993


In [9]:
class RCNNImageDataset(Dataset):
    IMAGE_WIDTH = 720
    IMAGE_HEIGHT = 480
    SCALE = 600 / IMAGE_HEIGHT
    def __init__(self, dataset_path, transforms=None):
        self.dataset_path = dataset_path
        self.img_path = os.path.join(dataset_path, 'images/render') # directory to images folder
        self.boxes_path = os.path.join(dataset_path, 'rcnn_bounding_boxes_final.csv') # directory to csv
        
        self.df = pd.read_csv(self.boxes_path) # bounding boxes df
        self.images = sorted(self.df['image'].unique()) # unique image name values: render0001.jpg, render9771.jpg
        
        self.transforms = transforms # transforms (if applicable)
                
    def __len__(self):
        return len(self.images)        
                
    def __getitem__(self, index):
        img_name = self.images[index]
        img_path = os.path.join(self.img_path, f"{img_name}")
        image = Image.open(img_path).convert('RGB')
        
        # Find bounding boxes and class_id for selected image
        img_df = self.df[self.df['image'] == img_name]
        boxes = img_df[['x_min', 'y_min', 'x_max', 'y_max']].values
        class_id = img_df['class_id'].values
        

        # transforms + Manual resizing

        # width, height = image.size # 720, 480
        # scaler = 7/15 # 336 x 224
        new_width = int(self.IMAGE_WIDTH * self.SCALE)
        new_height = int(self.IMAGE_HEIGHT * self.SCALE)
        
        image = image.resize((new_width, new_height))
        boxes = boxes * self.SCALE
        
        if self.transforms:
            image = self.transforms(image) # turns image into tensor
            
        # converts boxes and class_id to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(class_id, dtype=torch.int64)       
        
        # calculates area and iscrowd pytorch variables
        area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        iscrowd = torch.zeros((len(img_df),), dtype=torch.int64)
        
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([index]),
            "area": area,
            "iscrowd": iscrowd
        }
        
        # return image and target dict
        return image, target

Model Set Up!

In [10]:
# Model: PyTorch Faster R-CNN 

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

# Use CUDA in Kaggle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using: {device}")

# Data Processing
# dataset_path already defined

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = RCNNImageDataset(dataset_path, transform)

# Train: 70%, Val: 15%, Test: 15%
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

def collate(batch):
    return tuple(zip(*batch))

NUM_WORKERS = 2
BATCH_SIZE = 4
PREFETCH = 4
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=NUM_WORKERS,pin_memory=True, 
                          persistent_workers=False, collate_fn=collate, 
                          prefetch_factor=PREFETCH) 
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=NUM_WORKERS, pin_memory=True, 
                        persistent_workers=False, collate_fn=collate, 
                        prefetch_factor=PREFETCH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=NUM_WORKERS, pin_memory=True, 
                         persistent_workers=False, collate_fn=collate, 
                         prefetch_factor=PREFETCH)

Using: cuda


In [11]:
# Model Construction
model_weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=model_weights)
num_classes = 4 # 3 rock sizes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features # type: ignore

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.to(device)

# print(model) # List out RCNN layers
# loss_function = built-in, call while training

parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.AdamW(
    parameters,
    lr=2e-4,
    weight_decay=0.0005
)
# optimizer = optim.SGD(
#     parameters,
#     lr=0.005,
#     momentum=0.9,
#     weight_decay=0.0005
# )
scaler = torch.amp.GradScaler(device, enabled=True)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 218MB/s] 


In [12]:
def train_epoch(model, train_loader, optimizer, device, epoch, print_freq=100, accum_steps=1):
    model.train() # set model to train mode
    running_loss = 0.0
    
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}")

    optimizer.zero_grad()
    for batch_index, (images, targets) in progress_bar:
        # Move images/targets to device
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        with torch.amp.autocast('cuda'):
            loss_dict = model(images, targets)
            losses = sum(loss_dict.values()) / accum_steps
        # loss_dict = model(images, targets)
        # losses = sum((loss for loss in loss_dict.values() if torch.is_tensor(loss)), torch.tensor(0.0, device=device))
        
        # Backward pass
        scaler.scale(losses).backward()
        # losses.backward()
        # optimizer.step()
        
        # Update metrics
        if (batch_index + 1) % accum_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
        batch_loss = losses.item() * accum_steps
        running_loss += batch_loss
        progress_bar.set_postfix(loss=batch_loss)

        if (batch_index + 1) % print_freq == 0:
            avg_loss = running_loss / print_freq
            print(f"[Batch {batch_index + 1}/{len(train_loader)}] "
                  f"Loss: {avg_loss:.4f}")
            running_loss = 0.0 # reset running loss every {print_freq} batches
            
    return running_loss

In [13]:
def evaluate(model, test_loader, device):
    model.eval() # swithc to eval
    metric = MeanAveragePrecision() # mAP
    # total_loss = 0
    
    with torch.no_grad():
        for images, targets in val_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # no val loss in model.eval() mode
            # with torch.amp.autocast('cuda'):
            #     loss_dict = model(images, targets)
            #     losses = sum(loss_dict.values())
            #     total_loss += losses.item()
            # loss_dict = model(images, targets)
            # total_loss += sum(loss.item() if torch.is_tensor(loss) else float(loss) for loss in loss_dict.values())
    
            # get mAP predictions
            preds = model(images)
            metric.update(preds, targets)
            
    # avg_loss = total_loss / len(test_loader)
    metrics = metric.compute()

    return metrics # avg_loss

In [15]:
num_epochs = 10
best_mAP = 0.0

print("==========Training==========")
for epoch in range(1, num_epochs+1):
    print(f"\n==========Epoch: {epoch}==========")
    
    # Training
    train_loss = train_epoch(model, train_loader, optimizer, device, epoch)
    
    # Validating
    val_metrics = evaluate(model, val_loader, device)
    # print(f"Validation Loss: {val_loss:.4f}")
    print("Validation Metrics")
    print(f"mAP: {val_metrics['map']:.4f} | mAP_50: {val_metrics['map_50']:.4f} | mAP_75: {val_metrics['map_75']:.4f}")

    if val_metrics['map'] > best_mAP:
        best_mAP = val_metrics['map']
        torch.save(model.state_dict(), "best_model.pth")
        
        
# Testing
print("\n==========Testing==========")
test_metrics = evaluate(model, test_loader, device)
# print(f"Test Loss: {val_loss:.4f}")
print(f"mAP: {test_metrics['map']:.4f} | mAP_50: {test_metrics['map_50']:.4f} | mAP_75: {test_metrics['map_75']:.4f}")

==========Training==========

==========Epoch: 1==========


Epoch 1:  10%|█         | 100/994 [01:02<09:16,  1.61it/s, loss=0.283]

[Batch 100/994] Loss: 0.5135


Epoch 1:  20%|██        | 200/994 [02:05<08:15,  1.60it/s, loss=0.34] 

[Batch 200/994] Loss: 0.4020


...

[Batch 800/994] Loss: 0.1658


Epoch 10:  91%|█████████ | 900/994 [09:21<00:58,  1.60it/s, loss=0.143] 

[Batch 900/994] Loss: 0.1646


Epoch 10: 100%|██████████| 994/994 [10:20<00:00,  1.60it/s, loss=0.168] 


Validation Metrics
mAP: 0.4506 | mAP_50: 0.7045 | mAP_75: 0.5086

==========Testing==========
mAP: 0.4506 | mAP_50: 0.7045 | mAP_75: 0.5086


In [17]:
# meetrics decreasing man .... cuz the classes are shittt
# EDIT THE CLASSES


# Later add craters and other luanr objects